In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import polars as pl
from sklearn.datasets import make_regression

from timelake import TimeLake

In [2]:
# Params
n_hours = 24 * 30  # 30 days
n_features = 100

# Generate regression features
X, _ = make_regression(n_samples=n_hours, n_features=n_features)
timestamps = [datetime(2024, 1, 1) + timedelta(hours=i) for i in range(n_hours)]

# Create Polars DataFrame
df = pl.DataFrame(X, schema=[f"feature_{i}" for i in range(n_features)])
df = df.with_columns(pl.Series("date", timestamps))

In [3]:
PATH = Path("./my-test-timelake")

lake = TimeLake.create(
    path=PATH,
    df=df,
    timestamp_column="date",
)
df = lake.read()
df.shape

(720, 103)

In [4]:
lake.read(start_date="2024-01-01", end_date="2024-01-03").shape

(72, 103)

In [ ]:
df_update = df[0:10]
lake.upsert(df_update)

In [12]:
df = lake.read()
df.shape

(720, 103)

In [16]:
df

feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,…,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,date,inserted_at,date_day
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,datetime[μs],str,str
0.119926,0.275178,0.195489,0.159864,0.018352,-0.991033,1.192347,-0.219096,1.240684,2.335493,2.349359,-0.469634,1.510104,0.464336,-0.402361,2.166113,0.039769,0.832871,1.071859,-1.04632,-1.547964,0.079925,0.878948,2.090055,-0.799886,0.556151,0.075046,0.802693,-1.264853,-0.37707,-0.487008,-0.053728,0.892132,1.526157,0.231444,-0.35526,0.926419,…,0.149383,-0.562098,1.327944,-0.600175,0.003064,0.336039,0.295889,0.754015,1.480219,-0.665161,1.11667,0.010121,-0.556816,0.634186,-0.144419,-0.849553,0.712189,-0.30911,-0.226071,-1.046631,-1.028232,0.661688,-0.264409,1.981546,0.459908,-0.209635,-0.775015,0.117877,-0.491636,-1.979345,-0.205979,-2.013424,-1.465226,-0.476372,2024-01-15 14:00:00,"""2025-04-17T18:16:58.587932""","""2024-01-15"""
0.338291,0.108185,-0.412352,-0.790004,-0.756979,-1.324491,2.481011,0.858564,0.916955,-0.380794,-1.623711,0.245819,0.834382,-0.617785,1.536376,-0.327061,1.320967,-0.549384,-0.704132,0.07051,-0.240929,-0.051376,-1.031343,-0.93948,0.126853,-1.422607,0.572524,-0.543148,0.377686,0.149974,0.116917,0.239105,-0.506251,-0.548299,-0.043831,-0.096179,-1.849419,…,2.247015,0.908785,-1.151682,-0.734505,0.250367,-1.396002,-0.933688,0.138516,0.962714,-0.615709,-0.543536,1.02188,-0.922119,1.381878,-0.156537,0.239485,0.14866,-0.89695,-0.675406,1.11569,1.215247,1.626578,0.819034,0.407637,-0.014184,-1.560637,2.624124,1.479093,-0.617063,0.160537,-0.265706,-0.556065,-1.258252,-0.595168,2024-01-15 16:00:00,"""2025-04-17T18:16:58.587932""","""2024-01-15"""
-0.619267,-0.399609,-0.368632,1.460601,-0.918442,0.410297,-2.978407,0.524958,-0.267518,0.855575,0.653046,-0.836091,-0.896691,-1.521548,0.624379,0.63805,-1.086626,0.735449,-0.070526,-0.243964,0.945058,0.128053,-0.55201,-1.710433,-0.651977,0.466399,-0.957148,-0.689574,-0.648061,0.985517,-1.218978,1.577356,-0.434737,-0.250232,0.427606,0.465693,0.172637,…,-0.488085,-1.16916,1.408746,1.137914,-1.393379,-0.572487,1.393337,-0.839072,-1.88679,-1.962284,-0.247022,-0.503479,-0.122262,-0.931687,1.197809,-0.176488,-0.098953,-0.566606,-1.207429,-1.586686,0.287062,2.024653,-0.940682,-0.341898,-0.332674,-0.088957,0.288153,0.783845,-0.807188,0.142228,-0.189122,-0.45363,-2.153064,1.085369,2024-01-15 23:00:00,"""2025-04-17T18:16:58.587932""","""2024-01-15"""
0.457261,-0.753671,0.720189,0.568804,-0.329791,-0.287237,1.025979,1.138019,-0.594068,-0.485361,0.10559,0.74717,0.515755,-0.306867,0.734815,-0.132498,1.892897,0.36188,-0.310265,1.00066,-0.529174,1.099848,1.047144,1.14776,-1.217992,0.264382,-1.679362,0.187096,-1.05539,1.045645,0.228032,0.419729,-1.343534,-0.338295,0.686218,0.249934,-0.122615,…,-1.250595,-0.48611,0.100468,-0.749221,0.932656,0.30221,0.052478,-1.37879,-1.730779,-1.361796,0.403709,-1.457474,-1.007624,0.191312,0.190187,1.755039,-0.220086,-0.004223,0.423786,-0.211603,-0.978246,-0.851317,-1.378762,-0.514724,-0.114843,-1.470855,-1.065317,-0.876299,0.416559,-1.032616,-0.735569,0.02923,-0.985451,1.42